In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from GIN_jsspenv import GIN_JsspEnv
from agent import PPO
import warnings
warnings.filterwarnings('ignore')
import time
from collections import defaultdict
from collections import OrderedDict
from GIN_jsspenv import GIN_JsspEnv
from logger import Logger

actor_lr = 2e-5
critic_lr = 2e-5
gamma = 0.98
lmbda = 0.95
epochs = 10
eps = 0.2
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
def train_single_size_on(size, num_episodes_per_env=10, dir="logdir/"):
    env_name = f"train_single_size_on_{size}"
    exp_name = f"num_episodes_per_env={num_episodes_per_env}"
    logdir = dir + exp_name + '_' + env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
    logger = Logger(log_dir=logdir)
    logs = OrderedDict()
    
    num_jobs, num_machines = size
    generated_instances_file = f"DataGen/generatedData{num_jobs}_{num_machines}_Seed200.npy"
    policy_dir = f"saved_policies/train_single_size_on_{num_jobs}_{num_machines}_num_episodes_per_env={num_episodes_per_env}"
    agent = PPO(device, actor_lr, critic_lr, lmbda, epochs, eps, gamma)
    generated_instances = np.load(generated_instances_file)

    for i in range(len(generated_instances)):
        instance = generated_instances[i]
        time_mat, machine_mat = instance
        env = GIN_JsspEnv(processing_time_matrix=time_mat, machine_matrix=machine_mat)
        env.seed(0)
        rl_utils.train_on_policy_agent(env=env, agent=agent, num_episodes=num_episodes_per_env, logger=logger, logs=logs)

    agent.save(policy_dir)

In [3]:
sizes = [(6,6),(10,10),(15,10),(15,15),(20,10),(20,15),(20,20),(30,15),(30,20),(50,20),(100,20),(200,50)]
sizes = [(6,6)]
for size in sizes:
    train_single_size_on(size=size,num_episodes_per_env=10)

########################
logging outputs to  logdir/num_episodes_per_env=10_train_single_size_on_(6, 6)_20-04-2023_15-20-37
########################


KeyboardInterrupt: 